In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers
import json
import os

from openai import OpenAI

In [3]:
# CSV Datei laden
df = pd.read_csv('/content/data/Köhler_Iselborn_Rohdatensatz_Zenodo.csv')
df = df[df['Status'] == 'Completed'].copy()
df.drop(columns=['Fall', 'Status', 'Bearbeitungszeit(Sekunden)', 'ErklärungDatenverarbeitung', 'Q0', 'Q3'], inplace=True)
df.replace(-99, np.nan, inplace=True)
df.to_csv("clean_data.csv", index=False)
df

,Q1,Q2A,Q2B,Q2C,Q2D,Q2E,Q2F,Q2G,Q4A,Q4B,...,Q12I,Q13,Q14,Q15,Q16,Q17A,Q17B,Q17C,Q17D,Q17E
0,5,2,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,...,3.0,2.0,6.0,1.0,3.0,0.0,0.0,1.0,0.0,0.0
1,3,2,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,...,2.0,1.0,6.0,1.0,3.0,0.0,0.0,1.0,0.0,0.0
2,3,3,3.0,2.0,2.0,1.0,2.0,2.0,3.0,2.0,...,4.0,2.0,8.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0
4,3,2,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,...,2.0,1.0,5.0,1.0,3.0,0.0,0.0,1.0,0.0,0.0
7,2,2,3.0,1.0,2.0,3.0,3.0,3.0,3.0,3.0,...,5.0,1.0,5.0,1.0,3.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,3,3,3.0,1.0,1.0,3.0,3.0,3.0,2.0,3.0,...,3.0,1.0,8.0,1.0,9.0,0.0,0.0,1.0,0.0,0.0
532,2,3,3.0,2.0,1.0,3.0,3.0,2.0,3.0,2.0,...,5.0,2.0,7.0,1.0,3.0,0.0,0.0,1.0,0.0,1.0
533,3,2,3.0,1.0,1.0,3.0,3.0,3.0,1.0,3.0,...,1.0,2.0,12.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0
534,5,1,3.0,1.0,1.0,3.0,1.0,1.0,2.0,2.0,...,2.0,2.0,6.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0


In [13]:
#print(df.isna().sum())  # zeigt, ob alles korrekt ersetzt wurde
print("Anzahl der -99-Werte pro Spalte:")
print((df == -99).sum())
print("Gesamte Anzahl an -99-Werten im DataFrame:")
print((df == -99).sum().sum())


Anzahl der -99-Werte pro Spalte:
Q1      0
Q2A     0
Q2B     0
Q2C     0
Q2D     0
       ..
Q17A    0
Q17B    0
Q17C    0
Q17D    0
Q17E    0
Length: 83, dtype: int64
Gesamte Anzahl an -99-Werten im DataFrame:
0


In [6]:
client = OpenAI(api_key="")

In [19]:
# 1. CSV-Datei hochladen
uploaded_file = client.files.create(
    file=open("clean_data.csv", "rb"),
    purpose="assistants"
)


In [20]:
# 2. Assistant mit Code Interpreter + Dateizugriff
assistant = client.beta.assistants.create(
    name="SurveyStatAnalyzer",
    instructions="You analyze survey CSV files and compute per-column statistics (mean, std, min, max, mode, frequency) for each numeric column. Export the result to stat_summary.csv.",
    model="gpt-4o",
    tools=[{"type": "code_interpreter"}],
    tool_resources={
        "code_interpreter": {
            "file_ids": [uploaded_file.id]
        }
    }
)

In [21]:
import time

# 3. Thread erstellen
thread = client.beta.threads.create()

user_message = (
    "Please perform a full statistical analysis of the uploaded file `clean_data.csv`.\n"
    "For each numeric column, compute:\n"
    "- Column name\n"
    "- Mean\n"
    "- Standard deviation\n"
    "- Minimum\n"
    "- Maximum\n"
    "- Most frequent value (mode)\n"
    "- Frequency of that most frequent value\n"
    "Then export the result as a CSV file named `stat_summary.csv` with columns:\n"
    "Column, Mean, Std, Min, Max, Mode, Frequency. Provide a download link to the file."
)


# 4. Run starten (Datei wird automatisch über Assistant verwendet)
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions=user_message
)

# 5. Warten bis abgeschlossen
while True:
    run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run_status.status == "completed":
        break
    elif run_status.status == "failed":
        raise RuntimeError("Run failed.")
    time.sleep(2)

# 6. Ergebnis abrufen
messages = client.beta.threads.messages.list(thread_id=thread.id)
for message in messages.data:
    for content in message.content:
        if hasattr(content, "text"):
            print(content.text.value)

The statistical analysis for each numeric column has been completed, and the results have been summarized. The summary has been saved as a CSV file named `stat_summary.csv`.

You can download it from the link below:

[Download the summary CSV file](sandbox:/mnt/data/stat_summary.csv)
The dataset has been loaded successfully and contains 83 columns. Let's proceed to perform a statistical analysis for each numeric column and export the results as requested.


In [29]:
# Suche nach file_id über Message-Annotation (für vom Code Interpreter generierte Dateien)
file_id = None

for message in messages.data:
    for content in message.content:
        if content.type == "text":
            annotations = getattr(content.text, "annotations", [])
            for annotation in annotations:
                if annotation.type == "file_path" and "stat_summary.csv" in annotation.text:
                    file_id = annotation.file_path.file_id

if not file_id:
    raise ValueError("file_id für stat_summary.csv nicht gefunden.")

# Dateiinhalt mit Files API abrufen
file_stream = client.files.content(file_id)
file_bytes = file_stream.read()

# Lokal speichern
with open("stat_summary.csv", "wb") as f:
    f.write(file_bytes)

print("stat_summary.csv wurde erfolgreich gespeichert.")


stat_summary.csv wurde erfolgreich gespeichert.


# Data Generator with Statistics

In [30]:
stat_file = client.files.create(
    file=open("stat_summary.csv", "rb"),
    purpose="assistants"
)

# Erstelle neuen Assistant zur Datengenerierung basierend auf Statistikdatei
synth_assistant = client.beta.assistants.create(
    name="SyntheticDataGenerator",
    instructions="You generate synthetic survey data based on statistical summaries in CSV format.",
    model="gpt-4o",
    tools=[{"type": "code_interpreter"}],
    tool_resources={
        "code_interpreter": {
            "file_ids": [stat_file.id]
        }
    }
)

# Erstelle neuen Thread
synth_thread = client.beta.threads.create()

# Formuliere Prompt zur Generierung synthetischer Daten
synth_prompt = (
    "You are a synthetic data generation model. You are given a CSV file, which contains statistical summaries for a real-world survey dataset.\n"
    "Each row in the file corresponds to one survey question (column). Analyze the file to understand the statistical conditions for the to be generated survey data.\n\n"
    "Your task now is to generate 100 realistic synthetic survey responses in CSV format with the survey questions being the columns of the new generated file and each row a completed survey.\n"
    "Avoid generating the same survey answers. Be diverse and create new survey data for each row while ensuring following conditions:"
    "- All values fall within the specified min and max for each column\n"
    "- The overall distribution aligns with the mean and std\n"
    "- The mode appears slightly more frequently than other values where reasonable\n\n"
    "Return only the generated content as a CSV file (no explanation)."
)


In [31]:
# Starte Run zur Datengenerierung
synth_run = client.beta.threads.runs.create(
    thread_id=synth_thread.id,
    assistant_id=synth_assistant.id,
    instructions=synth_prompt
)

# Warte bis fertig
while True:
    run_status = client.beta.threads.runs.retrieve(thread_id=synth_thread.id, run_id=synth_run.id)
    if run_status.status == "completed":
        break
    elif run_status.status == "failed":
        raise RuntimeError("Datengenerierung fehlgeschlagen.")
    time.sleep(2)

# Nachrichten abrufen
synth_messages = client.beta.threads.messages.list(thread_id=synth_thread.id)
for message in synth_messages.data:
    for content in message.content:
        if hasattr(content, "text"):
            print(content.text.value)

The synthetic survey responses have been generated and saved to a CSV file. You can download it using the link below:

[Download the synthetic survey responses](sandbox:/mnt/data/synthetic_survey_responses.csv)
Based on the statistical summary provided, I will generate 100 synthetic survey responses that align with the given statistics for each survey question. The responses will be random yet conform to the constraints (minimum, maximum, mean, and standard deviation) and slightly favoring the mode as appropriate.

Let's generate the synthetic survey data.


In [32]:
# 1. Extrahiere file_id aus den Message-Annotations
synthetic_file_id = None

for message in synth_messages.data:
    for content in message.content:
        if content.type == "text":
            annotations = getattr(content.text, "annotations", [])
            for annotation in annotations:
                if annotation.type == "file_path" and "synthetic_survey_responses.csv" in annotation.text:
                    synthetic_file_id = annotation.file_path.file_id

if not synthetic_file_id:
    raise ValueError("file_id für synthetic_survey_responses.csv nicht gefunden.")

In [33]:
# 2. Dateiinhalt abrufen
response = client.files.content(synthetic_file_id)
file_bytes = response.read()

# 3. Als lokale Datei speichern
with open("synthetic_survey_responses.csv", "wb") as f:
    f.write(file_bytes)

# 4. Als DataFrame laden und anzeigen
df_synthetic = pd.read_csv("synthetic_survey_responses.csv")
df_synthetic.head()

,Q1,Q2A,Q2B,Q2C,Q2D,Q2E,Q2F,Q2G,Q4A,Q4B,...,Q12I,Q13,Q14,Q15,Q16,Q17A,Q17B,Q17C,Q17D,Q17E
0,3,3,2,1,1,2,3,2,3,2,...,2,1,7,1,4,1,0,0,0,0
1,2,2,2,1,1,1,2,3,3,3,...,2,1,5,4,4,0,0,1,1,0
2,3,2,3,2,2,3,3,3,3,3,...,3,1,11,1,2,1,0,1,1,0
3,3,1,3,1,2,2,3,3,3,1,...,3,1,10,3,2,0,0,1,0,0
4,3,2,2,1,2,1,2,3,3,1,...,4,2,9,6,2,0,0,0,1,1


In [34]:
# Nur numerische Spalten auswählen
df_numeric = df.select_dtypes(include='number')

# Erzeuge Statistik-Übersicht
stat_table = pd.DataFrame({
    "Mean": df_numeric.mean(),
    "Std": df_numeric.std(),
    "Min": df_numeric.min(),
    "Max": df_numeric.max(),
    "Most Frequent Value": df_numeric.mode().iloc[0],
    "Frequency": df_numeric.apply(lambda x: x.value_counts().iloc[0])
})

# Runde Zahlen für bessere Lesbarkeit
stat_table = stat_table.round(2)

# Zeige Tabelle
stat_table

,Mean,Std,Min,Max,Most Frequent Value,Frequency
Q1,3.10,1.03,1.0,5.0,3.0,216
Q2A,2.44,0.57,1.0,3.0,3.0,235
Q2B,2.66,0.52,1.0,3.0,3.0,332
Q2C,1.38,0.56,1.0,3.0,1.0,321
Q2D,1.34,0.57,1.0,3.0,1.0,340
...,...,...,...,...,...,...
Q17A,0.27,0.45,0.0,1.0,0.0,344
Q17B,0.01,0.09,0.0,1.0,0.0,468
Q17C,0.39,0.49,0.0,1.0,0.0,286
Q17D,0.20,0.40,0.0,1.0,0.0,376
